In [5]:
#importing dependencies
import pandas as pd
from sqlalchemy import create_engine
#from config import gkey, username, password
import requests
import json
import geojson

In [6]:
file_path = 'data/MSP_neighborhoods.csv'
MSP_df = pd.read_csv(file_path, encoding = "utf-8")

In [7]:
MSP_df.head()

,City,Neighborhood,Total population,Total population - Share - Margin of Error,Total population - Share,Total population - Margin of Error,Male,Male - Share - Margin of Error,Male - Share,Male - Margin of Error,...,Poverty 35-44 - Share,Poverty 35-44 - Margin of Error,Poverty 45-54,Poverty 45-54 - Share - Margin of Error,Poverty 45-54 - Share,Poverty 45-54 - Margin of Error,Poverty 55-64,Poverty 55-64 - Share - Margin of Error,Poverty 55-64 - Share,Poverty 55-64 - Margin of Error
0,Minneapolis,Armatage,5252,296,100%,5.63%,2590,212,49.32%,4.04%,...,0.33%,1.88%,0,NaN,0,2.05%,261,351,4.98%,6.69%
1,Minneapolis,Audubon Park,5318,311,100%,5.85%,2568,245,48.28%,4.60%,...,10.95%,5.21%,852,490.0,16.31%,9.39%,649,490,12.43%,9.39%
2,Minneapolis,Bancroft,3523,263,100%,7.47%,1766,170,50.14%,4.84%,...,7.88%,8.32%,189,188.0,5.38%,5.34%,626,364,17.78%,10.33%
3,St Paul,Battle Creek-Highwood,23757,961,100%,4.05%,12168,619,51.22%,2.61%,...,13.93%,5.01%,3806,1165.0,16.09%,4.93%,2212,903,9.35%,3.81%
4,Minneapolis,Beltrami,1177,143,100%,12.12%,607,110,51.54%,9.37%,...,1.68%,5.54%,240,165.0,20.56%,14.12%,346,200,29.71%,17.20%


In [8]:
# Convert age share columns to float and re-order into 5 different age groups
MSP_df['AgeBelow18Prct'] = (MSP_df['Under 5 years - Share'].str.rstrip('%').astype('float') / 100.0
                              + MSP_df['5-9 years - Share'].str.rstrip('%').astype('float') / 100.0
                              + MSP_df['10-14 years - Share'].str.rstrip('%').astype('float') / 100.0 
                              + MSP_df['15-17 years - Share'].str.rstrip('%').astype('float') / 100.0)

MSP_df['Age18To34Prct'] = (MSP_df['18-24 years - Share'].str.rstrip('%').astype('float') / 100.0
                              + MSP_df['25-34 years - Share'].str.rstrip('%').astype('float') / 100.0)

MSP_df['Age35To54Prct'] = (MSP_df['35-44 years - Share'].str.rstrip('%').astype('float') / 100.0
                              + MSP_df['45-54 years - Share'].str.rstrip('%').astype('float') / 100.0)

MSP_df['Age55To75Prct'] = (MSP_df['55-64 years - Share'].str.rstrip('%').astype('float') / 100.0
                              + MSP_df['65-74 years - Share'].str.rstrip('%').astype('float') / 100.0)

MSP_df['AgeAbove75Prct'] = (MSP_df['75-84 years - Share'].str.rstrip('%').astype('float') / 100.0
                              + MSP_df['85 years and older - Share'].str.rstrip('%').astype('float') / 100.0)

In [9]:
# Set percent string fields to float fields
MSP_df['PublicTransportPrct'] = MSP_df['Public transportation - Share'].str.rstrip('%').astype('float') / 100.0
MSP_df['WalkBiketoWorkPrct'] = MSP_df['Walked, biked, worked at home, or other - Share'].str.rstrip('%').astype('float') / 100.0
MSP_df['UnemploymentPrct'] = MSP_df['Unemployment rate - Share'].str.rstrip('%').astype('float') / 100.0

In [10]:
# Select columns to be used in our database
MSP_df_reduced = MSP_df[["Neighborhood", "City", "Total population",
    "Total households", "Median household income (2008-2012, 2012 dollars)", 
    "PublicTransportPrct", "WalkBiketoWorkPrct",
    "UnemploymentPrct", "AgeBelow18Prct", "Age18To34Prct", "Age35To54Prct",
    "Age55To75Prct", "AgeAbove75Prct"]]
MSP_df_reduced.head()

,Neighborhood,City,Total population,Total households,"Median household income (2008-2012, 2012 dollars)",PublicTransportPrct,WalkBiketoWorkPrct,UnemploymentPrct,AgeBelow18Prct,Age18To34Prct,Age35To54Prct,Age55To75Prct,AgeAbove75Prct
0,Armatage,Minneapolis,5252,2127,140363,0.0691,0.1130,0.0413,0.2381,0.2129,0.3249,0.1697,0.0544
1,Audubon Park,Minneapolis,5318,2268,69684,0.0773,0.1243,0.0345,0.1956,0.3027,0.2931,0.1880,0.0206
2,Bancroft,Minneapolis,3523,1517,71974,0.0916,0.1672,0.0483,0.2199,0.2803,0.2741,0.1912,0.0345
3,Battle Creek-Highwood,St Paul,23757,8107,52696,0.0464,0.0491,0.0571,0.2946,0.2735,0.2261,0.1675,0.0383
4,Beltrami,Minneapolis,1177,449,72188,0.1573,0.1103,0.0520,0.1810,0.4058,0.2582,0.1470,0.0081


In [ ]:
# Rename MSP_df columns to match target database
neighborhood_df = MSP_df_reduced.rename(columns={"Total population": "NeighborhoodPopulation", 
                       "Total households": "NeighborhoodHouseholds",
                      "Median household income (2008-2012, 2012 dollars)": "MedianIncome"})
neighborhood_df.head()

In [ ]:
# Calculate the center of each neighborhood in Minneapolis 
# by averaging boundary coordinates
neighborhoodName = []
lats = []
lngs = []
avgLats = []
avgLngs = []
with open('data/Minneapolis_neighborhoods.geojson') as f:
    gj = geojson.load(f)
    
    # grab features from each neighborhood
    for i in gj['features']:
        
        #grab neighborhood name
        neighborhood = i['properties']['BDNAME']
        neighborhoodName.append(neighborhood)
        
        #clear lat and lngs lists for each loop
        lats.clear()
        lngs.clear()
        
        #grab coordinates that make up neighborhood bounds
        coordinates = i['geometry']['coordinates'][0][0]
        for i in coordinates:
            lats.append(i[1])
            lngs.append(i[0])
        
        #calculate average coordinates from list of coordinates and store
        avgLat = sum(lats) / len(lats)
        avgLng = sum(lngs) / len(lngs)
        avgLats.append(avgLat)
        avgLngs.append(avgLng)

In [ ]:
# repeat the above process for St. Paul
with open('data/StPaul_neighborhoods.geojson') as f:
    gj = geojson.load(f)
    
    # grab features from each neighborhood
    for i in gj['features']:
        
        #grab neighborhood name
        neighborhood = i['properties']['name2']
        neighborhoodName.append(neighborhood)
        
        #clear lat and lngs lists for each loop
        lats.clear()
        lngs.clear()
        
        #grab coordinates that make up neighborhood bounds
        coordinates = i['geometry']['coordinates'][0][0]
        for i in coordinates:
            lats.append(i[1])
            lngs.append(i[0])
        
        #calculate average coordinates from list of coordinates and store
        avgLat = sum(lats) / len(lats)
        avgLng = sum(lngs) / len(lngs)
        avgLats.append(avgLat)
        avgLngs.append(avgLng)

In [ ]:
neighborhood_center_dict = {"Neighborhood" : neighborhoodName,
                            "Latitude" :  avgLats,
                            "Longitude" : avgLngs
}

In [ ]:
center_coord_df = pd.DataFrame(neighborhood_center_dict)
center_coord_df

In [ ]:
# list of place types we will be looking for within google places
typeList = ['supermarket', 'park', 'gym', 'restaurant', 'school', 'transit_station', 'church']

In [ ]:
# create dictionary for storing data
places = {
    "placeName": [],
    "placeType": [],
    "placeLat": [],
    "placeLng": []
}
places

In [ ]:
for type in typeList:
    # parameters for api call
    params = {
        "radius": 3000,
        "type": type,
        "key": gkey
    }

    #base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # use iterrows to iterate through pandas dataframe
    for index, row in center_coord_df.iterrows():
        lat = row['Latitude']
        long = row['Longitude']
        location = f"{lat}, {long}"

        # add keyword to params dict
        params['location'] = location

        # assemble url and make API request
        places_data = requests.get(base_url, params=params).json()
        places_info = places_data['results']
        
        # iterate through results to pull out the name, lat, and long of found places
        for i in places_info:
            places['placeName'].append(i['name'])
            places['placeType'].append(type)
            places['placeLat'].append(i['geometry']['location']['lat'])
            places['placeLng'].append(i['geometry']['location']['lng'])

In [ ]:
places_df = pd.DataFrame(places)
places_df

In [ ]:
places_df.count()

In [ ]:
# Delete duplicate values which may have been grabbed by overlapping radius calls
places_no_dup_df = places_df.drop_duplicates()
places_no_dup_df.count()

In [ ]:
# Connect to database
connection_string = f"{username}:{password}@localhost:5432/MSP_Neighborhoods"

# Create the engine
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# Checking connection by finding table names
engine.table_names()

In [ ]:
# Use "to_sql" function to load all transformed dfs' data into postgres

# Neighborhoods
neighborhood_df.to_sql(name='Neighborhoods', con=engine, if_exists='replace', index=True, index_label='NeighborhoodID')

# Places
places_no_dup_df.to_sql(name='Places', con=engine, if_exists='replace', index=True, index_label='placeID')